In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint
import time


# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


# Load the CSV
Csv_path = "../001_WeatherPy/002-Output/Weather_Data_DF.csv"
Weather_Data_DF = pd.read_csv(Csv_path)
Weather_Data_DF.dropna()
Weather_Data_DF.head()


# Create a list containing coordinates
coordinates = Weather_Data_DF[["Latitude", "Longitude"]].astype(float)
humidity = Weather_Data_DF["Humidity"].astype(float)

# Customize the size of the figure
figure_layout = {
    'width': '980px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '10px',
    'margin': '10 auto 10 auto'
}

# select a map type
fig = gmaps.figure(layout=figure_layout#, 
                   #map_type="HYBRID"
                  )


# Assign the marker layer to a variable
#markers = gmaps.marker_layer(coordinates)
markers = gmaps.heatmap_layer(coordinates)


# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=100,
                                 point_radius=1
                                )

# Add layer
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='10 auto 10 auto', padding='10px',…

In [2]:
plt.savefig('002-Output/Vacation_Map - Humidity.png')


<Figure size 432x288 with 0 Axes>

In [3]:

#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

#Best_Cities_DF = Weather_Data_DF.loc[Weather_Data_DF["Max Temp"] < 80, :]
#Best_Cities_DF = Weather_Data_DF.loc[Weather_Data_DF["Max Temp"] > 70, :]
#Best_Cities_DF = Weather_Data_DF.loc[Weather_Data_DF["Wind Speed"] < 10, :]
#Best_Cities_DF = Weather_Data_DF.loc[Weather_Data_DF["Cloudiness"] == 0, :]


Best_Cities_DF = Weather_Data_DF.drop(Weather_Data_DF[(Weather_Data_DF["Max Temp"] < 150) 
                                                     | (Weather_Data_DF["Wind Speed"] > 5) 
                                                     | (Weather_Data_DF["Cloudiness"] != 0)
                                                     | (Weather_Data_DF["Humidity"] > 30)
                                                      
                                                    ].index
                                    )

In [4]:
Best_Cities_DF

,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Max Temp,Wind Speed,Cloudiness
87,Atar,2381334,MR,20.52,-13.05,290.60,26,290.60,1.18,0
122,Adrar,2381972,MR,20.50,-10.07,295.09,19,295.09,3.92,0
144,Calvinia,3369174,ZA,-31.47,19.78,287.01,24,287.01,1.87,0
188,Sabha,2212775,LY,27.04,14.43,291.80,23,291.80,2.73,0
204,Aranos,3358666,NaN,-24.13,19.12,296.19,18,296.19,3.95,0
304,Keetmanshoop,3356264,NaN,-26.58,18.13,295.15,24,295.15,1.50,0


In [5]:
print("Total number of cities is : " + str(len(Best_Cities_DF)))

Total number of cities is : 6


In [6]:
#Hotel Map
#Store into variable named hotel_df.
hotel_df = Best_Cities_DF

In [7]:
Best_Cities_DF["Hotel"] = ""

Best_Cities_DF.head()

,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Max Temp,Wind Speed,Cloudiness,Hotel
87,Atar,2381334,MR,20.52,-13.05,290.60,26,290.60,1.18,0,
122,Adrar,2381972,MR,20.50,-10.07,295.09,19,295.09,3.92,0,
144,Calvinia,3369174,ZA,-31.47,19.78,287.01,24,287.01,1.87,0,
188,Sabha,2212775,LY,27.04,14.43,291.80,23,291.80,2.73,0,
204,Aranos,3358666,NaN,-24.13,19.12,296.19,18,296.19,3.95,0,


In [8]:
#**********************TEST************************
#**************************************************

#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

params = {
    "location": "23.47, 73.30",
    "Keyword" : "international airport",
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#base_url = "https://maps.googleapis.com/maps/api/place/details/json?"


# run a request using our params dictionary
response = requests.get(base_url, params=params)
#print(response)


# convert response to json
response_Json = response.json()
#print(response_Json)
print(json.dumps(response_Json, indent=4, sort_keys=True))



{
    "html_attributions": [],
    "next_page_token": "CqQCGgEAAKcvtE9U8P8digXXIN7bO6hCODeWHn-VdVU_GxJWbsN3McPSa-kNik7vS2Hg0dAW1SvU2b4-MxREabUeymwUwrTo08-jaXNBUTgubElV4fdS_1kSX8KsMf2A38jaIrCAjDO1yoPUxV6EEVRxB9n_VWZIUoQlIRz_VPa3z0W1kop19FwPJZNwR2JjQ-ENjfOua6XwKW3tDy1erwuGdVFUsXA2KQaT9ppn9awJ2QjP1GtBRDUb2agfoo_smZ5bXGw_uTvkdWlDK9arq7T0PZuUALnOXDXJaZEqx_REI5DQJ2O1xgQ-41gSH8oFu4WROXMd9uFZf_BJEVPr2kKwDrlP-GpiOAqsi7nl_-IswqefavYuM4XB_YDZ8NP4lh9M1echBRIQL5hFavg4KvSI2BmW1p6UmxoUTrvgSy7eKdxPQ8fRDZs4ifAxBgI",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 23.464311,
                    "lng": 73.29875779999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 23.4848003,
                        "lng": 73.3194749
                    },
                    "southwest": {
                        "lat": 23.4401753,
                        "lng": 73.28142629999999
          

In [9]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.


# create a params dict that will be updated with new city each iteration
params = {"radius": 5000,
          "type": "Hotel",
          "keyword" : "Hotel",
          "key": g_key
         }

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#base_url = "https://maps.googleapis.com/maps/api/place/details/json?"



# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():

    Lat = row['Latitude']
    Long = row['Longitude']
    #CityID = row["City_ID"]
    
    # update address key value
    params['location'] = f"{Lat},{Long}"    
    #pprint(params)

    # make request
    Response = requests.get(base_url, params=params)
    #print(Response)
          
    # convert to json
    Response_JSON = Response.json()
    #pprint(Response_JSON)
    
    # extract results
    results = Response_JSON['results']
    pprint(results)

    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel"] = Response_JSON["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    



[{'geometry': {'location': {'lat': 20.4908504, 'lng': -13.0629797},
               'viewport': {'northeast': {'lat': 20.49198337989272,
                                          'lng': -13.06119037010728},
                            'southwest': {'lat': 20.48928372010728,
                                          'lng': -13.06389002989272}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': '88818bb171dd6f51f39d8f90910660b957ea5ba1',
  'name': 'Odar kanawal',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 2322,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/100455968816246200066">A '
                                    'Google User</a>'],
              'photo_reference': 'CmRaAAAAlU7ticStzVD-ncQIWQOmgyq4Sf0_4AAcXf3_o_PXVpk2SaAvzgaGhsdnMhMMaj0fWTynd1yBTJuruc8TrIG3Wl3r1ZlxyMjf-pn9QxToFeMmjIOS-3gGDztzP5Mv4NOTEhCUieF_w7i7EOiSyaZVU684GhT4MnuuwEOu29DwXg_sE1rg1bbkyA

[]
Missing field/result... skipping.
[{'geometry': {'location': {'lat': -31.4716378, 'lng': 19.7795305},
               'viewport': {'northeast': {'lat': -31.47030577010728,
                                          'lng': 19.78091077989272},
                            'southwest': {'lat': -31.47300542989272,
                                          'lng': 19.77821112010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': '448c34fd9edff5964384ee90a3be593d3bec8632',
  'name': 'Tarantula Guesthouse',
  'opening_hours': {'open_now': False},
  'photos': [{'height': 2448,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/114475857444068787713">A '
                                    'Google User</a>'],
              'photo_reference': 'CmRaAAAA-V7SL1urc1W3dfEwr0_4sBNCy7HrI_1klmiuWCX1kebdmGMZ4lzkP4VgzR9L4UT4qFZkyx7OJaPMH7_rdXyLCcjb9O-PqmMvhv_Z83cl9jAn5xz7HEKYKlBmoZsnV3lBEhCsSv1E

[{'geometry': {'location': {'lat': 27.0599264, 'lng': 14.4222471},
               'viewport': {'northeast': {'lat': 27.06089152989273,
                                          'lng': 14.42339792989272},
                            'southwest': {'lat': 27.05819187010728,
                                          'lng': 14.42069827010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': 'dcf3b1e1f5bc8b5507e2133697fd204267a35164',
  'name': 'فندق المواجيد',
  'place_id': 'ChIJHQiOPTP7xhMRMwnksTgIreY',
  'plus_code': {'compound_code': '3C5C+XV Sabhā, Libya',
                'global_code': '7FVP3C5C+XV'},
  'rating': 0,
  'reference': 'ChIJHQiOPTP7xhMRMwnksTgIreY',
  'scope': 'GOOGLE',
  'types': ['lodging', 'point_of_interest', 'establishment'],
  'user_ratings_total': 0,
  'vicinity': 'مخازن الأسواق سابقاً, سبها-حي عبدالكافي مقابل, Sabhā'},
 {'geometry': {'location': {'lat': 27.0284718, 'lng': 14.4280995},
               'viewport': {'northeast': {

[{'geometry': {'location': {'lat': -26.5818852, 'lng': 18.13412},
               'viewport': {'northeast': {'lat': -26.58057172010727,
                                          'lng': 18.13548327989272},
                            'southwest': {'lat': -26.58327137989272,
                                          'lng': 18.13278362010728}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': '2ee80e329b0a894667f4d2f105a8d877db8c0f41',
  'name': 'Quiver Inn Guesthouse',
  'photos': [{'height': 1080,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/112072170158063816882">Andre '
                                    'Terblanche</a>'],
              'photo_reference': 'CmRaAAAAHn8QnK8TvLmrAxKlGmU3ttgcLUMFU1J8oq5kqHkW9yvSQwJvMl5EBHUusPbMiBACZTREJLSoJ9Tuc4gNLocioL7A5Q1rgdBGdtvaRrNQel9QOHY4JYRR_eqr6jh3OULDEhB0GPYidMdIDerlKei07AtaGhREMNChqQRd3uayRF71QWQ53SlJ9w',
              'width': 192

In [10]:
# Visualize to confirm lat lng appear
hotel_df.head()

,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Max Temp,Wind Speed,Cloudiness,Hotel
87,Atar,2381334,MR,20.52,-13.05,290.60,26,290.60,1.18,0,Odar kanawal
122,Adrar,2381972,MR,20.50,-10.07,295.09,19,295.09,3.92,0,
144,Calvinia,3369174,ZA,-31.47,19.78,287.01,24,287.01,1.87,0,Tarantula Guesthouse
188,Sabha,2212775,LY,27.04,14.43,291.80,23,291.80,2.73,0,فندق المواجيد
204,Aranos,3358666,NaN,-24.13,19.12,296.19,18,296.19,3.95,0,Aranos Hotel


In [11]:
# replace empty values with NaN
hotel_df = hotel_df.replace("", np.nan, regex=True)
hotel_df

,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Max Temp,Wind Speed,Cloudiness,Hotel
87,Atar,2381334,MR,20.52,-13.05,290.60,26,290.60,1.18,0,Odar kanawal
122,Adrar,2381972,MR,20.50,-10.07,295.09,19,295.09,3.92,0,NaN
144,Calvinia,3369174,ZA,-31.47,19.78,287.01,24,287.01,1.87,0,Tarantula Guesthouse
188,Sabha,2212775,LY,27.04,14.43,291.80,23,291.80,2.73,0,فندق المواجيد
204,Aranos,3358666,NaN,-24.13,19.12,296.19,18,296.19,3.95,0,Aranos Hotel
304,Keetmanshoop,3356264,NaN,-26.58,18.13,295.15,24,295.15,1.50,0,Quiver Inn Guesthouse


In [12]:
# dropna
narrowed_city_df = hotel_df.dropna()
narrowed_city_df

,City,City_ID,Country,Latitude,Longitude,Temperature,Humidity,Max Temp,Wind Speed,Cloudiness,Hotel
87,Atar,2381334,MR,20.52,-13.05,290.60,26,290.60,1.18,0,Odar kanawal
144,Calvinia,3369174,ZA,-31.47,19.78,287.01,24,287.01,1.87,0,Tarantula Guesthouse
188,Sabha,2212775,LY,27.04,14.43,291.80,23,291.80,2.73,0,فندق المواجيد


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '980px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
markers = gmaps.marker_layer(locations,
                             info_box_content=hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

In [15]:
plt.savefig('002-Output/Vacation_Map.png')


<Figure size 432x288 with 0 Axes>